In [94]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math
import numpy as np

In [95]:
characters = pd.read_csv("../Clean data/clean_character_data.csv")
movies = pd.read_csv("../Clean data/clean_movie_data.csv")
inputs = pd.read_csv("../Clean data/inputs.csv")

column_names = list(inputs.columns)
print(column_names)

['Unnamed: 0', 'Wiki ID', 'Movie name', 'Release date', 'Freebase ID', 'Revenue', 'Runtime', 'IMDb rating', 'Wins', 'Nominations', 'Release year', 'Languages', 'Genres', 'Countries', 'Thriller-onehot', 'Science Fiction-onehot', 'Horror-onehot', 'Adventure-onehot', 'Action-onehot', 'Drama-onehot', 'Comedy-onehot', 'Romance Film-onehot', 'Musical-onehot', 'Fantasy-onehot', 'Family Film-onehot', 'Crime Fiction-onehot', 'Indie-onehot', 'World cinema-onehot', 'Mystery-onehot', 'Black-and-white-onehot', 'Crime Thriller-onehot', 'English Language-onehot', 'Tamil Language-onehot', 'Hindi Language-onehot', 'Malayalam Language-onehot', 'German Language-onehot', 'Japanese Language-onehot', 'Korean Language-onehot', 'Spanish Language-onehot', 'Italian Language-onehot', 'French Language-onehot', 'Standard Mandarin-onehot', 'Standard Cantonese-onehot', 'Bengali Language-onehot', 'Cantonese-onehot', 'Telugu language-onehot', 'Russian Language-onehot', 'Portuguese Language-onehot', 'Urdu Language-oneh

In [96]:
#Garder uniquement les features que l'on va utiliser
inputs = inputs[['Revenue', 'Runtime','Release year', 'Thriller-onehot', 'Science Fiction-onehot', 'Horror-onehot', 'Adventure-onehot', 'Action-onehot', 'Drama-onehot', 'Comedy-onehot', 'Romance Film-onehot', 'Musical-onehot', 'Fantasy-onehot', 'Family Film-onehot', 'Crime Fiction-onehot', 'Indie-onehot', 'World cinema-onehot', 'Mystery-onehot', 'Black-and-white-onehot', 'Crime Thriller-onehot', 'English Language-onehot', 'Tamil Language-onehot', 'Hindi Language-onehot', 'Malayalam Language-onehot', 'German Language-onehot', 'Japanese Language-onehot', 'Korean Language-onehot', 'Spanish Language-onehot', 'Italian Language-onehot', 'French Language-onehot', 'Standard Mandarin-onehot', 'Standard Cantonese-onehot', 'Bengali Language-onehot', 'Cantonese-onehot', 'Telugu language-onehot', 'Russian Language-onehot', 'Portuguese Language-onehot', 'Urdu Language-onehot', 'Arabic Language-onehot', 'Tagalog language-onehot', 'Swedish Language-onehot', 'United States of America-onehot', 'United Kingdom-onehot', 'India-onehot', 'Canada-onehot', 'Germany-onehot', 'Japan-onehot', 'South Korea-onehot', 'Argentina-onehot', 'France-onehot', 'Spain-onehot', 'Ireland-onehot', 'China-onehot', 'Russia-onehot', 'Australia-onehot', 'Italy-onehot', 'Mexico-onehot', 'New Zealand-onehot', 'Brazil-onehot', 'Belgium-onehot', 'Denmark-onehot', 'Sweden-onehot', 'Netherlands-onehot', 'Philippines-onehot', 'Primary Actor height', 'Primary Actor age at release', 'Primary M-onehot', 'Primary American Indian/Alaska Native', 'Primary Black/African American', 'Primary East Asian', 'Primary Other', 'Primary South Asian', 'Primary White', 'Secondary Actor height', 'Secondary Actor age at release', 'Secondary M-onehot', 'Secondary American Indian/Alaska Native', 'Secondary Black/African American', 'Secondary East Asian', 'Secondary Other', 'Secondary South Asian', 'Secondary White']]
inputs_cleaned = inputs.dropna()
#Mettre les revenus en logarithmique mais pas sur qu'on fasse ça
inputs_cleaned["Revenue"] = np.log10(inputs['Revenue'])
inputs_cleaned["Revenue"]

C:\Users\foale\AppData\Local\Temp\ipykernel_12144\1520541976.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputs_cleaned["Revenue"] = np.log10(inputs['Revenue'])


3      4.400172
4      8.323606
5      5.464302
6      6.870857
7      7.701357
         ...   
781    5.525941
782    7.991495
783    5.636146
786    8.148664
789    7.897492
Name: Revenue, Length: 345, dtype: float64

In [97]:
#TEST pour voir si diminuer le nombre des features change quelque chose
inputs_cleaned = inputs_cleaned[["Revenue",'Runtime','Release year','Primary Actor age at release']]

In [98]:
# On mets les features dans X et les Revenus dans Y
X = inputs_cleaned.drop('Revenue', axis=1)  # Features
y = inputs_cleaned['Revenue']  # Target variable

#Splitting du data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Fit le model
model = xgb.XGBRegressor(objective='reg:squarederror')
model.fit(X_train, y_train)

#Des test pour voir si ça fonctionnes
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("mse",math.sqrt(mse))
variance = y_test.std()
print("Std", variance)

mse 1.120261340365381
Std 1.0807508032988435


In [99]:
for element in y_pred:
    print(element)

6.19586
7.970877
7.0405097
8.723306
8.111265
8.128713
7.6760945
8.046755
8.250245
7.5157757
6.9777937
7.7905073
6.686413
7.3147836
8.129866
7.3542714
8.134911
8.126074
6.546028
8.54825
7.605305
6.621642
5.417147
8.288813
8.190514
7.7172484
6.0140743
5.0593286
7.9055805
7.3550034
7.556227
7.143161
7.6072574
8.358439
7.393347
6.767843
6.174884
8.339433
8.215553
8.020191
7.928767
5.3437285
8.382769
8.347367
8.015833
7.8384337
8.616517
7.312259
6.284432
7.743581
6.6424327
7.7218738
7.797047
7.265251
7.506384
6.973168
8.36659
7.710731
7.201526
6.388555
7.5718627
7.689198
7.4589305
6.926328
7.393529
6.456599
8.016608
7.7538567
7.3540206


In [100]:
for element in y_test:
    print(element)

max_value = np.max(y_test)
min_value = np.min(y_test)

print("Highest value in test:", max_value)
print("Lowest value:", min_value)

max_value = np.max(y_pred)
min_value = np.min(y_pred)

print("Highest value in pred:", max_value)
print("Lowest value:", min_value)

7.95746365080208
8.028605385598578
7.382684227932023
8.52212367767101
7.99149674629179
6.947361213830245
7.30802302030645
8.640531935730905
7.25242803289006
6.939847387277027
4.464015448334903
7.570541573260241
6.434799257403802
6.9087022033822265
6.677782147114173
7.7267892537380405
7.254060280398518
7.963330044253438
8.064923053208828
7.466428080606911
8.06744043426585
8.529585705474332
7.786242470953514
7.39287095109613
7.51534240910984
7.36108244481259
4.182569343771263
7.188752705183384
7.669761144788209
8.298698673742937
8.538515960911903
5.495778704685626
6.205745645570014
7.671164229273236
8.400402968105096
8.056128828575229
3.049213232631098
7.43763876081744
8.24057207483624
8.455955653170195
6.57263477273383
5.067431678293587
5.636145900651357
7.4141779367538945
6.870857374632109
7.718426486257907
8.269075390293347
7.130047972264228
7.724803582642681
8.024754322455387
7.457791978893631
8.253093955354109
6.734261705851501
7.6390921225115855
8.050834136936626
8.012079961108732
